In [1]:
! pip install transformers
! pip install sentencepiece
import sentencepiece
import torch
from torch import nn
from torch.nn import BCEWithLogitsLoss
import torchsummary as summary

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm, trange
import gc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00


In [2]:
torch.cuda.empty_cache()

In [3]:
gc.collect()

4

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/')


df=pd.read_csv('final_labelled_enron.csv')
df=df[~(df['content']=='nan')]
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)


Mounted at /content/drive


In [5]:
'''
df=pd.read_csv('final_labelled_enron.csv')
df['content']=df['content'].astype(str)
df['subject']=df['subject'].astype(str)
df['result'].unique()
'''

"\ndf=pd.read_csv('final_labelled_enron.csv')\ndf['content']=df['content'].astype(str)\ndf['subject']=df['subject'].astype(str)\ndf['result'].unique()\n"

In [6]:
df['result']=df['result']-1
df['result'].unique()

array([3, 1, 2, 5, 0, 4, 6])

In [7]:
df['content']=df['subject']+' '+df['content']
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head()

,message_id,subject,content,result
0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,more about you a little bird told me about thi...,3
1,<17858209.1075857584947.JavaMail.evans@thyme>,lng on the road,lng on the road trucking lng to california the...,3
2,<14187877.1075857584924.JavaMail.evans@thyme>,amazoncom password assistance,amazoncom password assistance greeting from am...,1
3,<31194989.1075857585233.JavaMail.evans@thyme>,deferral enrollment 2001,deferral enrollment 2001 the annual deferral p...,3
4,<32084772.1075857584968.JavaMail.evans@thyme>,lng question,lng question rus a couple of question check th...,3


In [8]:
df_3=df[(df['result']==3) & (df['content'].str.len()<350)]
df=df[~(df['result']==3)]
df=pd.concat([df,df_3])
df

,message_id,subject,content,result
2,<14187877.1075857584924.JavaMail.evans@thyme>,amazoncom password assistance,amazoncom password assistance greeting from am...,1
5,<29043641.1075857584989.JavaMail.evans@thyme>,wassup,wassup hey freako how have you been im just ta...,1
10,<7789871.1075857584856.JavaMail.evans@thyme>,holiday party,holiday party i know it will be close to impos...,2
12,<4081281.1075857631018.JavaMail.evans@thyme>,mother day,mother day thank you for the beautiful flower ...,1
13,<8847335.1075857631040.JavaMail.evans@thyme>,bnp paribas commodity future ng marketwatch fo...,bnp paribas commodity future ng marketwatch fo...,5
...,...,...,...,...
13553,<33080058.1075845335601.JavaMail.evans@thyme>,re western wholesale activity gas power conf c...,re western wholesale activity gas power conf c...,3
13579,<9002886.1075852513161.JavaMail.evans@thyme>,privileged and confidential attorney client co...,privileged and confidential attorney client co...,3
13580,<20906757.1075852522540.JavaMail.evans@thyme>,confidential attorney work client privilege as...,confidential attorney work client privilege as...,3
13590,<30078399.1075852530017.JavaMail.evans@thyme>,re western wholesale activity gas power conf c...,re western wholesale activity gas power conf c...,3


In [9]:
# XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        self.labels=df['result']
        self.text=[tokenizer(text,padding='max_length',max_length=512,truncation=True,return_tensors="pt") for text in df['content']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df[['message_id','subject','content']], df['result'], test_size=0.25, stratify=df['result'])

In [12]:
X_train['result']=y_train
X_test['result']=y_test
df_train=X_train
df_val=X_test
df_test=0

In [13]:
df_train.reset_index(inplace=True)
df_train=df_train.drop('index',axis=1)

df_val.reset_index(inplace=True)
df_val=df_val.drop('index',axis=1)
'''
df_test.reset_index(inplace=True)
df_test=df_test.drop('index',axis=1)
'''
print(len(df_train),len(df_val))

10018 3340


In [14]:
df_val

,message_id,subject,content,result
0,<26139694.1075857756075.JavaMail.evans@thyme>,re structured energy update,re structured energy update howard can you rem...,1
1,<18092760.1075857684057.JavaMail.evans@thyme>,interview on 1801 manu asthana,interview on 1801 manu asthana greg dont forge...,3
2,<17897591.1075857753495.JavaMail.evans@thyme>,newsprint stats note on eolclickpaper,newsprint stats note on eolclickpaper figure o...,0
3,<12382995.1075841676466.JavaMail.evans@thyme>,a new way to pay,a new way to pay rick causey recently announce...,6
4,<7996998.1075861666109.JavaMail.evans@thyme>,trv notification ng price pl 11212001,trv notification ng price pl 11212001 the repo...,6
...,...,...,...,...
3335,<6280232.1075841643364.JavaMail.evans@thyme>,tfs,tfs should there be a fee on this deal thanks,0
3336,<15267340.1075857594923.JavaMail.evans@thyme>,re,re eat shit john j pm john cc football bet eac...,3
3337,<26691844.1075852531386.JavaMail.evans@thyme>,erratum to srp settlement proposal privileged ...,erratum to srp settlement proposal privileged ...,0
3338,<8304993.1075846160550.JavaMail.evans@thyme>,lous 50000,lous 50000 lou you may be maxed out on frank n...,3


In [15]:
# XLNet classifier architecture, with 7 output classes
class XLNetClassifier(nn.Module):
    def __init__(self, dropout=0.3):
        super(XLNetClassifier, self).__init__()
        self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased', return_dict=True)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(self.xlnet.config.hidden_size, 7)
        torch.nn.init.kaiming_uniform_(self.linear.weight, nonlinearity='relu')
        self.relu = nn.ReLU()

    def forward(self, input_ids, attention_mask):

        outputs = self.xlnet(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Get the CLS token representation
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer


In [17]:
# change runtype to GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [18]:
# hyperparameters
EPOCHS = 300
model = XLNetClassifier()
LR = 0.00001

In [19]:
df_train=df_train.reset_index()
df_train.drop('index',axis=1,inplace=True)
df_train

,message_id,subject,content,result
0,<1461291.1075857602432.JavaMail.evans@thyme>,re,re sec row,3
1,<4273989.1075860879869.JavaMail.evans@thyme>,get keyex list from ll,get keyex list from ll task assignment status ...,0
2,<1135589.1075841703661.JavaMail.evans@thyme>,re 424 checkout,re 424 checkout both of these just went in let...,0
3,<16455972.1075860790736.JavaMail.evans@thyme>,chris foster,chris foster is in,0
4,<20115903.1075842099175.JavaMail.evans@thyme>,re ipe brent language,re ipe brent language sara this is greek to me...,2
...,...,...,...,...
10013,<7547550.1075841617021.JavaMail.evans@thyme>,re enpower counterparty name change,re enpower counterparty name change wow i bet ...,2
10014,<33211186.1075845556340.JavaMail.evans@thyme>,re trading tape,re trading tape mark we can make this even sim...,3
10015,<24289974.1075857595225.JavaMail.evans@thyme>,re,re yep brian communication am john cc fangming...,3
10016,<6315150.1075857656386.JavaMail.evans@thyme>,re hjk,re hjk it one of those theoretically great tra...,1


In [20]:
# display BERT layers
n=0
for x in model.state_dict():
    n=n+1
    print(x)
n

xlnet.mask_emb
xlnet.word_embedding.weight
xlnet.layer.0.rel_attn.q
xlnet.layer.0.rel_attn.k
xlnet.layer.0.rel_attn.v
xlnet.layer.0.rel_attn.o
xlnet.layer.0.rel_attn.r
xlnet.layer.0.rel_attn.r_r_bias
xlnet.layer.0.rel_attn.r_s_bias
xlnet.layer.0.rel_attn.r_w_bias
xlnet.layer.0.rel_attn.seg_embed
xlnet.layer.0.rel_attn.layer_norm.weight
xlnet.layer.0.rel_attn.layer_norm.bias
xlnet.layer.0.ff.layer_norm.weight
xlnet.layer.0.ff.layer_norm.bias
xlnet.layer.0.ff.layer_1.weight
xlnet.layer.0.ff.layer_1.bias
xlnet.layer.0.ff.layer_2.weight
xlnet.layer.0.ff.layer_2.bias
xlnet.layer.1.rel_attn.q
xlnet.layer.1.rel_attn.k
xlnet.layer.1.rel_attn.v
xlnet.layer.1.rel_attn.o
xlnet.layer.1.rel_attn.r
xlnet.layer.1.rel_attn.r_r_bias
xlnet.layer.1.rel_attn.r_s_bias
xlnet.layer.1.rel_attn.r_w_bias
xlnet.layer.1.rel_attn.seg_embed
xlnet.layer.1.rel_attn.layer_norm.weight
xlnet.layer.1.rel_attn.layer_norm.bias
xlnet.layer.1.ff.layer_norm.weight
xlnet.layer.1.ff.layer_norm.bias
xlnet.layer.1.ff.layer_1.weig

208

In [21]:
# freeze first 8 layers
n=0
for param in model.parameters():
    n=n+1
    param.requires_grad = False
    if n==(201-68):
        break

In [24]:
# change datatypes of input data
df_train['message_id']=df_train['message_id'].astype(str)
df_train['subject']=df_train['subject'].astype(str)
df_train['content']=df_train['content'].astype(str)

df_val['message_id']=df_val['message_id'].astype(str)
df_val['subject']=df_val['subject'].astype(str)
df_val['content']=df_val['content'].astype(str)

In [25]:
# input_id.shape

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)
    # mini batching
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=30)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=30)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            n=0
            for train_input, train_label in tqdm(train_dataloader):
                train_label = train_label.to(device) # to cuda GPU
                mask = train_input['attention_mask'].squeeze(1).to(device) # attention mask
                input_id = train_input['input_ids'].squeeze(1).to(device)

                l1_loss=0

                # for L1 regularization
                a=0
                reg_loss = 0
                for param in model.parameters():
                    a=a+1
                    if a >=201-68:
                        reg_loss += torch.norm(param, 1)

                factor = 0.00001 #lambda for L1 regularization
                l1_loss=factor * reg_loss # L1 loss

                # model output
                output = model(input_id, mask)

                # loss value
                batch_loss = criterion(output, train_label)+l1_loss
                total_loss_train += batch_loss.item()

                # train accuracy
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                # backpropogation
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            # for validation accuracy
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].squeeze(1).to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    # validation output
                    output = model(input_id, mask)

                    # validation loss value
                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    # validation accuracy
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')



train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 334/334 [24:02<00:00,  4.32s/it]


Epochs: 1 | Train Loss:  0.384                 | Train Accuracy:  0.399                 | Val Loss:  0.052                 | Val Accuracy:  0.443


100%|██████████| 334/334 [24:01<00:00,  4.32s/it]


Epochs: 2 | Train Loss:  0.373                 | Train Accuracy:  0.496                 | Val Loss:  0.046                 | Val Accuracy:  0.501


100%|██████████| 334/334 [24:01<00:00,  4.32s/it]


...

100%|██████████| 334/334 [24:01<00:00,  4.32s/it]


Epochs: 7 | Train Loss:  0.341                 | Train Accuracy:  0.815                 | Val Loss:  0.048                 | Val Accuracy:  0.603


100%|██████████| 334/334 [24:01<00:00,  4.32s/it]


Epochs: 8 | Train Loss:  0.338                 | Train Accuracy:  0.838                 | Val Loss:  0.051                 | Val Accuracy:  0.612


 14%|█▍        | 48/334 [03:26<20:35,  4.32s/it]

In [ ]:
'''
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

evaluate(model, df_test)
'''

In [ ]:
# save model file
# torch.save(model.state_dict(), 'XLNet_model_freeze.pth')

In [ ]:
df